# Домашнее задание 3, Лупашин Евгений, БПМИ184

## 0. Чтение данных

In [1]:
! pip install category-encoders
! pip install sweetviz
! pip install dtale
import pandas as pd
import numpy as np
import sweetviz as sv
import dtale
%matplotlib inline
from category_encoders.woe import WOEEncoder

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_acc = pd.read_csv("19_accept.csv")
df_rej = pd.read_csv("19_reject.csv")

c:\users\pipek\downloads\wpy64-3810\python-3.8.1.amd64\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
# Первые строки заявок в accept
print('Размер accept:', df_acc.shape[0])
print('Размер reject:', df_rej.shape[0])
df_acc.head(10)

Размер accept: 32554
Размер reject: 10693


,target,INCOME_TYPE,EDUCATION_,CUSTOMER_CATEGORY_,income,DEPENDANT_NUMBER,IS_EMPL,COUNT_ACTIVE_CREDIT_NO_CC,CNT_MNTH_FROM_LAST_PMNT,age,experience,inquiry_14_day,inquiry_21_day,count_mnth_act_passport,UID
0,0,OTHER,elementary,NaN,140000.000,0.000,0,0.000,34.219,38.497,4.060,NaN,NaN,10.056,7
1,0,OTHER,advanced,Corporate,21611.000,0.000,1,NaN,NaN,33.333,1.712,NaN,NaN,7.973,15
2,0,OTHER,elementary,VIP,146841.000,0.000,1,0.000,30.531,36.531,0.578,0.000,0.000,7.738,17
3,0,OTHER,elementary,Corporate,102915.000,0.000,1,2.000,1.174,54.064,1.459,0.000,0.000,8.889,21
4,0,2NDFL,elementary,Corporate,51581.000,0.000,1,NaN,NaN,29.050,0.581,NaN,NaN,6.200,26
5,0,OTHER,elementary,Corporate,39713.000,0.000,1,0.000,1362.505,25.327,0.848,1.000,1.000,5.215,28
6,0,2NDFL,elementary,NaN,97414.000,0.000,0,2.000,-0.596,32.515,10.178,1.000,1.000,3.666,35
7,0,2NDFL,high,NaN,27697.000,0.000,0,0.000,1.506,26.874,3.584,0.000,0.000,6.573,39
8,0,OTHER,advanced,Corporate,25448.000,0.000,1,NaN,NaN,55.429,7.661,NaN,NaN,9.923,42
9,0,OTHER,elementary,Corporate,102207.000,0.000,0,1.000,0.049,34.241,8.874,2.000,2.000,10.282,50


## 1. Какая доля 1 в выборке "accept"?

In [3]:
df_acc[df_acc.target == 1].shape[0] / df_acc.shape[0]

0.021410579345088162

## 2. Статистики по переменным

Необходимо рассчитать для всех интервальных переменных следующее:
 - Доля пропущенных значений 
 - Медиана
 - Среднее
 - Среднеквадратическое отклонение
 - Есть ли аномальные значения, выбросы? 
 - Information Value
 
 
Необходимо рассчитать для всех категориальных переменных следующее:
 - Мода
 - Доля пропущенных значений
 - Information Value
 - Есть ли выбросы, аномальные значений
 
 
На всё, кроме выбросов и IV ответит отчёт, который генерируется в следующей клетке (`report.html`). По нажатию на признак видны его основные показатели. Выбросы далее тоже будем оценивать исходя из отчёта. Information Value для всех переменных и для всех датасетов посчитаем после обучения модели для разметки reject

In [4]:
report = sv.compare([df_acc, "AcceptDF"], [df_rej, "RejectDF"], "target")
report.show_html('report.html')

                                             |                                             | [  0%]   00:00 ->…

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Что насчёт выбросов?

Есть люди с доходом, превышающим 500000. Очевидно, таких людей мы будем относить к одной какой-то категории `богачи`, запомнили.

CNT_MNTH_FROM_LAST_PMNT ведёт себя довольно странно. Если я правильно понимаю, что это должно означать "количество месяцев с последнего платежа", то там не должно быть значений меньше 0. А они есть. И есть очень большие значения, больше 1000.

experience, тоже почему-то бывает отрицательным, а бывает и очень большим, но это нормально, вероятно.

count_mnth_act_passport, опять есть отрицательные значения

В категориальных признаках аномалий не обнаружено

## 4. Построить логистическую регрессию только на одобренных заявках с преобразованными переменными WoE. Какое значение GINI? F1 мера?

Сначала заполним пропуски на основе отчёта. income заполним нулями, возможно у людей, про которых мы не знаем их доход, он не очень велик. experience заполним нулём как самой популярной категорией, да и пропусков там мало. Пропуски в других категориях просто выделим в отдельную категорию

In [5]:
df_acc.income.fillna(value=0, inplace=True)
df_rej.income.fillna(value=0, inplace=True)

df_acc.experience.fillna(value=0, inplace=True)
df_rej.experience.fillna(value=0, inplace=True)

df_all = pd.concat([df_acc, df_rej])

df_all.income                  = pd.qcut(df_all.income, q=10, labels=None)
df_all.CNT_MNTH_FROM_LAST_PMNT = pd.qcut(df_all.CNT_MNTH_FROM_LAST_PMNT, q=10, labels=None)
df_all.age                     = pd.qcut(df_all.age, q=10, labels=None)
df_all.experience              = pd.qcut(df_all.experience, q=10, labels=None)
df_all.count_mnth_act_passport = pd.qcut(df_all.count_mnth_act_passport, q=10, labels=None)
df_all.UID                     = pd.qcut(df_all.UID, q=10, labels=None)

dummies = pd.get_dummies(df_all, columns=df_all.columns.drop('target'), dummy_na=True)
df_acc = dummies.iloc[:df_acc.shape[0], :]
df_rej = dummies.iloc[df_rej.shape[0]:, :]

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

X = df_acc.copy()
y = X.target
X = X.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=228179, stratify=y)

In [7]:
model = RandomForestClassifier(max_features="log2", max_depth=10)

In [8]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
print('F1:', f1_score(model.predict(X_test), y_test))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]) # roc_auc_score(model.predict(X_test), y_test)
Gini = 2 * roc_auc - 1
print('ROC AUC:', roc_auc)
print('Gini:', Gini)

F1: 0.1342281879194631
ROC AUC: 0.8362677089966445
Gini: 0.672535417993289


## 5. Провести анализ Reject Inference. Какая доля отказанных заявок?

Теперь обучим на всей выборке и разметим reject по такому порогу, чтобы доля отказанных заявок была примерно равна оной в accept

In [10]:
model.fit(X, y)
df_pred = df_rej.copy().drop('target', axis=1)
df_pred['target'] = model.predict_proba(df_pred)[:, 1]
df_pred['target'] = np.where(df_pred['target'] > 0.06, 1, 0)

In [20]:
print('доля отказанных заявок в reject:', df_pred[df_pred.target == 1].shape[0] / df_pred[df_pred.target == 0].shape[0])

доля отказанных заявок в reject: 0.02820504721897603


In [21]:
print('доля отказанных заявок в accept:', df_acc[df_acc.target == 1].shape[0] / df_acc[df_acc.target == 0].shape[0])

доля отказанных заявок в accept: 0.021879021879021878


## 2 и 3 (продолжение)
Наконец после разметки данных посчитаем WoE и Information Value для всех переменных и для всех датасетов

In [13]:
# Полезные функции

# source: https://gist.github.com/dradecic/52d8b2b2213dd3d46f4b75f85c1183f2
def calculate_woe_iv(dataset, feature, target, iv=False, sort=False):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    if sort:
        dset = dset.sort_values(by='WoE')
        
    if iv:
        iv = dset['IV'].sum()
        return dset, iv
    else:
        return dset

# source: https://www.kaggle.com/puremath86/iv-woe-starter-for-python
def woe_iv(dataframe, feature, target='target'):
    return (pd.crosstab(dataframe[feature], dataframe[target], normalize='columns')
        .assign(WoE = lambda df: np.log(df[1] / df[0])).replace({np.inf: 0, -np.inf: 0})
        .assign(IV = lambda df: np.nansum(df.WoE * (df[1] - df[0])))
    )

In [14]:
df_all_pred = df_acc.append(df_pred)
for col in list(df_all_pred.columns)[1:]:
    try:
        print(woe_iv(df_all_pred[cols], col), "\n")
    except:
        pass

## 6. Построить логистическую регрессию на всех заявках с преобразованными переменными WoE

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_all_pred.drop('target', axis=1),
                                                    df_all_pred.target,
                                                    test_size=0.2,
                                                    random_state=4533357)

In [16]:
%%time
WOE_encoder = WOEEncoder()
X_train = WOE_encoder.fit_transform(X_train, y_train)
X_test = WOE_encoder.transform(X_test)

Wall time: 9 ms


c:\users\pipek\downloads\wpy64-3810\python-3.8.1.amd64\lib\site-packages\category_encoders\utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [18]:
model.fit(X_train, y_train)

c:\users\pipek\downloads\wpy64-3810\python-3.8.1.amd64\lib\site-packages\sklearn\linear_model\_logistic.py:937: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
print('F1:', f1_score(model.predict(X_test), y_test))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
Gini = 2 * roc_auc - 1
print('ROC AUC:', roc_auc)
print('Gini:', Gini)

F1: 0.20708446866485014
ROC AUC: 0.8841218431039979
Gini: 0.7682436862079958


## 7. Какую модель вы рекомендуете для внедрения в продуктивную среду? Дать развернутое пояснение

Вторая модель получилась лучше по всем использованным метрикам, конечно, среди этих двух я порекомендую её. В целом можно было подойти более подробно к изучению выбросов, разбиению переменные на бины и последующему объединению похожих бинов, подбору гиперпараметров моделей и много чему ещё, но модель и так получилась достаточно хорошей, на мой взгляд